In [13]:
from avro.datafile import DataFileReader 
from avro.io import DatumReader
import json
import csv
import os

# Path for each subject:
#2
avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject2/1-1-03_1709406714.avro"
output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject2"
# #3
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject3/1-1-01_1709243558.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject3"
# #4 has multiple
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject4/1-1-04_1709408529.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject4"
# #5
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject5/1-1-02_1709326201.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject5"
# #6
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject6/1-1-06_1709573603.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject6"
# #7
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject7/1-1-05_1709480935.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject7"
# #8 has multiple
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject8/1-1-09_1709660046.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject8"
# #9
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject9/1-1-07_1709589779.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject9"
# #10
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject10/1-1-10_1709676666.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject10"
# #11 has multiple
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject11/1-1-08_1709654596.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject11"
# #12 has multiple
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject12/1-1-12_1709771553.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject12"
# #13
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject13/1-1-11_1709733543.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject13"
#14
# avro_file_path = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/data_sorted/subject14/1-1-13_1709922553.avro"
# output_dir = "/Users/sjtok/misophoniaAR/misoAR/Empatica_MisoAR_pilot/csv_extracted/subject14"

## Read Avro file
reader = DataFileReader(open(avro_file_path, "rb"), DatumReader()) 
schema = json.loads(reader.meta.get('avro.schema').decode('utf-8')) 
data= next(reader)

## Print the Avro schema
print(schema)

## Export sensors data to csv files
# Accelerometer
acc = data["rawData"]["accelerometer"]
timestamp = [round(acc["timestampStart"] + i * (1e6 / acc["samplingFrequency"]))
            for i in range(len(acc["x"]))]
# Convert ADC counts in g
delta_physical = acc["imuParams"]["physicalMax"] - acc["imuParams"]["physicalMin"] 
delta_digital = acc["imuParams"]["digitalMax"] - acc["imuParams"]["digitalMin"] 
x_g = [val * delta_physical / delta_digital for val in acc["x"]]
y_g = [val * delta_physical / delta_digital for val in acc["y"]]
z_g = [val * delta_physical / delta_digital for val in acc["z"]]
with open(os.path.join(output_dir, 'accelerometer.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["unix_timestamp", "x", "y", "z"])
    writer.writerows([[ts, x, y, z] for ts, x, y, z in zip(timestamp, x_g, y_g, z_g)])

# Gyroscope
gyro = data["rawData"]["gyroscope"]
timestamp = [round(gyro["timestampStart"] + i * (1e6 / gyro["samplingFrequency"]))
            for i in range(len(gyro["x"]))]
with open(os.path.join(output_dir, 'gyroscope.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["unix_timestamp", "x", "y", "z"])
    writer.writerows([[ts, x, y, z] for ts, x, y, z in zip(timestamp, gyro["x"], gyro["y"], gyro["z"])])
# Eda
eda = data["rawData"]["eda"]
timestamp = [round(eda["timestampStart"] + i * (1e6 / eda["samplingFrequency"]))
            for i in range(len(eda["values"]))]
with open(os.path.join(output_dir, 'eda.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["unix_timestamp", "eda"])
    writer.writerows([[ts, eda] for ts, eda in zip(timestamp, eda["values"])])
# Temperature
tmp = data["rawData"]["temperature"]
timestamp = [round(tmp["timestampStart"] + i * (1e6 / tmp["samplingFrequency"]))
            for i in range(len(tmp["values"]))]
with open(os.path.join(output_dir, 'temperature.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["unix_timestamp", "temperature"])
    writer.writerows([[ts, tmp] for ts, tmp in zip(timestamp, tmp["values"])])
# Tags
tags = data["rawData"]["tags"]
with open(os.path.join(output_dir, 'tags.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["tags_timestamp"])
    writer.writerows([[tag] for tag in tags["tagsTimeMicros"]])
# BVP
bvp = data["rawData"]["bvp"]
timestamp = [round(bvp["timestampStart"] + i * (1e6 / bvp["samplingFrequency"]))
            for i in range(len(bvp["values"]))]
with open(os.path.join(output_dir, 'bvp.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["unix_timestamp", "bvp"])
    writer.writerows([[ts, bvp] for ts, bvp in zip(timestamp, bvp["values"])])
# Systolic peaks
sps = data["rawData"]["systolicPeaks"]
with open(os.path.join(output_dir, 'systolic_peaks.csv'), 'w', newline='') as f:
    writer = csv.writer(f) 
    writer.writerow(["systolic_peak_timestamp"]) 
    writer.writerows([[sp] for sp in sps["peaksTimeNanos"]])
# Steps
steps = data["rawData"]["steps"]
timestamp = [round(steps["timestampStart"] + i * (1e6 / steps["samplingFrequency"]))
            for i in range(len(steps["values"]))]
with open(os.path.join(output_dir, 'steps.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["unix_timestamp", "steps"])
    writer.writerows([[ts, step] for ts, step in zip(timestamp, steps["values"])])

{'doc': 'Session file for Empatica devices.', 'fields': [{'doc': 'Version of the AVRO file schema.', 'name': 'schemaVersion', 'type': {'fields': [{'doc': 'Major version.', 'name': 'major', 'type': 'int'}, {'doc': 'Minor version.', 'name': 'minor', 'type': 'int'}, {'doc': 'Patch version.', 'name': 'patch', 'type': 'int'}], 'name': 'Version', 'type': 'record'}}, {'doc': 'Version of the device firmware.', 'name': 'fwVersion', 'type': 'empatica.format.avro.Version'}, {'doc': 'Version of the device hardware.', 'name': 'hwVersion', 'type': 'empatica.format.avro.Version'}, {'doc': 'Version of the processing algorithm.', 'name': 'algoVersion', 'type': 'empatica.format.avro.Version'}, {'default': 0, 'doc': 'Delta [s] from UTC [s] time at the location of the participant.', 'name': 'timezone', 'type': 'int'}, {'doc': 'Information of the participant enrolled.', 'name': 'enrollment', 'type': {'fields': [{'doc': 'Participant identifier.', 'name': 'participantID', 'type': 'string'}, {'doc': 'Site ide